In [13]:
import rasterio  
import rasterio.mask
import geopandas as gps
import os
from shapely.geometry import Point, Polygon,box

In [2]:
auxiliary_data_dir=r'D:\lakemapping\0_auxiliary_data\auxiliary_dataset.gdb'
# grwl_path = os.path.join(auxiliary_data_dir, 'GRWL') 
# OSM_path =  os.path.join(auxiliary_data_dir, 'OSM_water') 
raw_prediction_dir=r'J:\result836\model0121\tif'
output_dir=r'J:\result836\model0121\aftermask'

In [19]:
for index in range(1,10):
    raw_prediction_image=rasterio.open(os.path.join(raw_prediction_dir,'pre_max_iew100_{}.tif'.format(index)))
    bbox = box(*raw_prediction_image.bounds)
    meta=raw_prediction_image.meta
    meta.update({"compress": 'lzw'})
    grwl_river_patch = gps.read_file(auxiliary_data_dir,layer='GRWL_DNRiver' ,bbox=bbox) # data from GRWL_mask_V01.01 product with DN = 255 (River) (see https://doi.org/10.5281/zenodo.1297434).
    ocean_patch = gps.read_file(auxiliary_data_dir,layer='oceanline_polygon', bbox=bbox) 
    mask_patch = grwl_river_patch._append(ocean_patch)
    mask_patch_geojson = mask_patch.geometry.values 
 # perform river mask and ocean mask
    if len(mask_patch_geojson) > 0:
        modelpredtictions_masked, out_meta = rasterio.mask.mask(raw_prediction_image, mask_patch_geojson, all_touched=True, invert=True)
        print(f'Mask finished: {index} / 10')
    else:
        modelpredtictions_masked = raw_prediction_image.copy()
        print(f'No masks detected, output original image instead: {i}')
    with rasterio.open(output_dir+'/prediction_{}_masked.tif'.format(index), "w", **meta) as prediction_masked:
        prediction_masked.write(modelpredtictions_masked)


Mask finished: 1 / 10
Mask finished: 2 / 10
Mask finished: 3 / 10
Mask finished: 4 / 10
Mask finished: 5 / 10
Mask finished: 6 / 10
Mask finished: 7 / 10
Mask finished: 8 / 10
Mask finished: 9 / 10
